Finetuning hyperparameters

ROC AUC

Modes: 2 (BN, MC)
Rs: 4 (0,3,6,9)

LRs: 4 (2E-5, 1E-5, 2E-6, 1E-6)
Epochs: 5 (5, 10, 15, 20, #) [mc needs more epochs]

Metrics: 3 (Accuracy+F1, Accuracy+Recall, Recall+F1)
Batch: 2 (16, 32)
Loss: 3 (Default, Custom loss function with weighting, BCE weighted loss)

Confusion matrices

In [2]:
import os, json
cwd = os.getcwd() # should be base directory of repository
import time
import torch
from datasets import DatasetDict, Dataset
output_dir = cwd+"/outputs/fting"
input_dir = cwd+"/inputs/"

In [10]:
# uncomment if you dont already have the dataset dictionaries generated
'''
from classifier.finetune import load_labelintdcts, create_dsdict, create_om_dsdict
from classifier.run_classifiers import group_duplicates, remove_duplicates, dcno_to_sentlab
with open(input_dir+"/19Jan25_firstdatarev.json","r", encoding="utf-8") as f:
        dcno_json = json.load(f)
with open(input_dir+"/27Jan25_query_checked.json","r", encoding="utf-8") as f:
    qry_json = json.load(f)
sents1, labels1 = dcno_to_sentlab(dcno_json)
sents2, labels2 = dcno_to_sentlab(qry_json)
# merge original and augmented datasets
sents2.extend(sents1)
labels2.extend(labels1)
all_sents, all_labs = remove_duplicates(group_duplicates(sents2,labels2,thresh=90))
int2label_dct, label2int_dct = load_labelintdcts()
sims = range(10)
#create_dsdict(all_sents, all_labs, label2int_dct, amt=sims, save=True, output_dir=input_dir)
create_om_dsdict(all_sents, all_labs, label2int_dct, amt=sims, save=True, output_dir=input_dir)
'''

'\nfrom classifier.finetune import load_labelintdcts, create_dsdict, create_om_dsdict\nfrom classifier.run_classifiers import group_duplicates, remove_duplicates, dcno_to_sentlab\nwith open(input_dir+"/19Jan25_firstdatarev.json","r", encoding="utf-8") as f:\n        dcno_json = json.load(f)\nwith open(input_dir+"/27Jan25_query_checked.json","r", encoding="utf-8") as f:\n    qry_json = json.load(f)\nsents1, labels1 = dcno_to_sentlab(dcno_json)\nsents2, labels2 = dcno_to_sentlab(qry_json)\n# merge original and augmented datasets\nsents2.extend(sents1)\nlabels2.extend(labels1)\nall_sents, all_labs = remove_duplicates(group_duplicates(sents2,labels2,thresh=90))\nint2label_dct, label2int_dct = load_labelintdcts()\nsims = range(10)\n#create_dsdict(all_sents, all_labs, label2int_dct, amt=sims, save=True, output_dir=input_dir)\ncreate_om_dsdict(all_sents, all_labs, label2int_dct, amt=sims, save=True, output_dir=input_dir)\n'

First we're going to train our baseline model and get a view of the training progress in a ROC analysis

In [3]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

def plot_roc(labels, probs, mode, rstate, output_dir, int2label):
    if mode=="bn":
        fpr, tpr, thresholds = roc_curve(labels, probs[:, 1])
        roc_auc = auc(fpr, tpr)
        #
        plt.figure()
        plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
        plt.plot([0, 1], [0, 1], linestyle='--', color='teal')
        plt.xlabel("FPR")
        plt.ylabel("TPR")
        plt.title(f"ROC Curve {mode} {rstate}")
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{output_dir}/ROC_{mode}_{rstate}.png")
        #plt.show()
    elif mode=="mc":
        n_classes = probs.shape[1]
        y_test_bin = label_binarize(labels, classes=range(n_classes))
        # Compute ROC for each class
        fpr = {}
        tpr = {}
        roc_auc = {}
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], probs[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Plot all
        plt.figure()
        colors = ['firebrick', 'darkorange', 'gold', 'yellowgreen', "deepskyblue", "slateblue"]
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], color=colors[i], label=f"Class {int2label[i]} (AUC = {roc_auc[i]:.2f})")
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel("FPR")
        plt.ylabel("TPR")
        plt.title(f"ROC Curve {mode} {rstate}")
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{output_dir}/ROC_{mode}_{rstate}.png")
        #plt.show()
    else:
        n_classes = probs.shape[1]
        y_test_bin = label_binarize(labels, classes=range(n_classes))
        # Compute ROC for each class
        fpr = {}
        tpr = {}
        roc_auc = {}
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], probs[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        # Plot all
        plt.figure()
        colors = ['firebrick', 'darkorange', 'gold', 'yellowgreen', "deepskyblue", "slateblue", "mediumorchid"]
        for i in range(n_classes):
            plt.plot(fpr[i], tpr[i], color=colors[i], label=f"Class {int2label[i]} (AUC = {roc_auc[i]:.2f})")
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel("FPR")
        plt.ylabel("TPR")
        plt.title(f"ROC Curve {mode} {rstate}")
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{output_dir}/ROC_{mode}_{rstate}.png")
        #plt.show()

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer, AdamW, get_linear_schedule_with_warmup
import evaluate
from sklearn.metrics import roc_curve
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import shuffle
import numpy as np
import torch.nn as nn
from classifier.finetune import finetune_roberta, load_labelintdcts
import gc
import wandb
''''''

class WeightedTrainer(Trainer):
    def __init__(self, *args, loss_ratio=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_ratio = loss_ratio
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):#):#
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Define class weights and loss
        weights = torch.tensor(self.loss_ratio).to(logits.device)
        loss_fct = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits, labels)
        # loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def finetune_roberta(datasetdct, int2label, label2int, mode, model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=f"{os.getcwd()}/outputs/models", hyperparams=False, report_to="none", span=False):
    '''
    '''
    if not hyperparams:
        hyperparams = {
            "epochs":10, 
            "r":9,
            "lr":2e-5,
            "batch_size":16,
            "loss":False,
            "oversampling":False
            }
    epochs = hyperparams["epochs"]
    rstate = hyperparams["r"]
    lr = hyperparams["lr"]
    batch_size = hyperparams["batch_size"]
    loss_ratio = hyperparams["loss"]
    ovs_ratio = hyperparams["oversampling"]
    start = time.time()
    num_lbs = len(list(int2label))
    print(f'\nLoading model {model_name}\n')
    print("Tokenizing")
    if not span:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
    elif span == "bn":
        tokenizer = AutoTokenizer.from_pretrained("../../inputs/polianna_models/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_bn_e2_r9.pt")
    else:
        tokenizer = AutoTokenizer.from_pretrained("../../inputs/polianna_models/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_mc_e2_r9.pt")
    def preprocess_function(examples):
        return tokenizer(examples["text"], truncation=True, padding=True).to(dev)
    tokenized_test = datasetdct["test"].map(preprocess_function, batched=True)
    if not ovs_ratio:
        tokenized_train = datasetdct["train"].map(preprocess_function, batched=True)
    else:
        train_sents = datasetdct["train"]["text"]
        train_labels = datasetdct["train"]["label"]
        ros = RandomOverSampler(sampling_strategy='auto', random_state=rstate)
        train_texts_resampled, train_labels_resampled = ros.fit_resample(np.array(train_sents).reshape(-1, 1), np.array(train_labels))
        train_texts_resampled, train_labels_resampled = shuffle(train_texts_resampled, train_labels_resampled, random_state=rstate)
        flattened_texts = list(train_texts_resampled.flatten())
        conv_dct = {"text":flattened_texts, "label":train_labels_resampled}
        conv_ds = Dataset.from_dict(conv_dct)
        tokenized_train = conv_ds.map(preprocess_function, batched=True)
    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    recall = evaluate.load("recall")
    metric_log = []
    def calc_metrics(pred):
        predictions, labels = pred
        predictions = np.argmax(predictions, axis=1)
        metrics = {
            "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
            #"f1": f1.compute(predictions=predictions, references=labels, average="weighted" if mode=="mc" else "binary")["f1"],
            "f1": f1.compute(predictions=predictions, references=labels, average="weighted")["f1"],
            "recall": recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]
        }
        metric_log.append(metrics)
        return metrics
    print("Loading model")
    #
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_lbs,id2label=int2label, label2id=label2int).to(dev)
    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        seed=9,
        num_train_epochs=epochs,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="best",
        logging_strategy="epoch",
        optim="adamw_torch",
        load_best_model_at_end=True,
        report_to= report_to,
        run_name=f"{output_dir.split('_')[-2]}{rstate}" if report_to == "wandb" else "X"
    )
    #
    if loss_ratio:
        trainer = WeightedTrainer(
            loss_ratio=loss_ratio,
            model=model,
            args=training_args,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_test,
            processing_class=tokenizer,
            compute_metrics=calc_metrics,
            #scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)
        )
    else:
        trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_train,
                eval_dataset=tokenized_test,
                processing_class=tokenizer,
                compute_metrics=calc_metrics,
            )
    print("Training")
    trainer.train()
    train_losses = [log["loss"] for log in trainer.state.log_history if "loss" in log]
    eval_losses = [log["eval_loss"] for log in trainer.state.log_history if "eval_loss" in log]
    print("Saving")
    model_fn = f"{model_name.split('/')[-1]}_{mode}_e{epochs}_r{rstate}.pt"
    trainer.save_model(output_dir+f"/{model_fn}")
    #
    tokenized_ho = datasetdct["holdout"].map(preprocess_function, batched=True)
    predictions = trainer.predict(tokenized_ho)
    logits = predictions.predictions
    labels = predictions.label_ids
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=1).numpy()
    #
    plot_roc(labels, probs, mode, rstate, output_dir, int2label)
    metric_log.append({"train_loss":train_losses})
    metric_log.append({"eval_loss": eval_losses})
    metrics = calc_metrics((logits, labels))
    with open(output_dir+f"/{model_fn}/metrics.json", "w", encoding="utf-8") as f:
        json.dump(metric_log, f, ensure_ascii=False, indent=4)
    end = time.time()
    print(metrics)
    print(f"\nSaved {model_name.split('/')[-1]}_{mode}_e{epochs}_r{rstate}.")
    print(f'\nDone in {round((end-start)/60,2)} min')
    return metrics


Here we create all of our finetuned models

In [ ]:
!pip install wandb
!wandb login ################################

^C


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\allie\_netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
os.environ["WANDB_PROJECT"]="fting_incentives"
os.environ["WANDB_LOG_MODEL"]="checkpoint"
os.environ["WANDB_WATCH"]="false"

int2label_dct, label2int_dct = load_labelintdcts()
metriclog = {}
exps = [0,3,6]#range(10)#[6]#[0,3,6,9]#
torch.cuda.empty_cache()
gc.collect()

# mode: binary, spna_step: binary
# mode: binary, span_Step: multiclass
# mode: multiclass, span_Step: binary
# mode: multiclass, san_Step: multiclass
# mode: omni, span_Step: binary
# mode: omni, san_Step: multiclass
for r in exps:
    letter = "I"
    mode = "bn"
    output_dir = cwd+f"/outputs/fting_{letter}_{mode}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    ds = DatasetDict.load_from_disk(f"{input_dir}/ds_{r}_{mode}")
    hyper = {
        "epochs":3, 
        "r":r, 
        "lr":2E-5,
        "batch_size":16,
        "loss":[0.175,8.81,2.57,2.78,25.3,10.7,3.32],
        "oversampling":None,
        "span_step":"binary"
        }
    torch.cuda.empty_cache()
    gc.collect()
    wandbrun = wandb.init(config=hyper, group=letter, name=f"{letter}{r}", reinit='create_new')
    metrics = finetune_roberta(ds, int2label_dct[mode], label2int_dct[mode], mode, model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper, report_to="wandb", span="bn")
    wandbrun.finish()
    metriclog[f'{mode}_{r}'] = metrics
    hyp_rpt = {mode:hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)
'''
for r in exps:
    output_dir = cwd+"/outputs/fting_J_om"
    ds = DatasetDict.load_from_disk(f"{input_dir}/ds_{r}_om")
    hyper = {
        "epochs":10, 
        "r":r, 
        "lr":2E-5,
        "batch_size":16,
        "loss":None,
        "oversampling":"auto"
        }
    torch.cuda.empty_cache()
    gc.collect()
    wandbrun = wandb.init(config=hyper, group="J", name=f"J{r}", reinit='create_new')
    metrics = finetune_roberta(ds, int2label_dct["om"], label2int_dct["om"], "om", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper, report_to="wandb")
    wandbrun.finish()
    metriclog[f'om_{r}'] = metrics
    hyp_rpt = {"om":hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)
'''
print(metriclog)


Loading model sentence-transformers/paraphrase-xlm-r-multilingual-v1

Tokenizing
Loading model


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-xlm-r-multilingual-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall
1,1.654800,1.279524,0.764085,0.774290,0.764085
2,0.943000,0.964896,0.813380,0.817319,0.813380
3,0.620500,0.922653,0.806338,0.815575,0.806338


wandb: Adding directory to artifact (c:\Users\allie\Documents\GitHub\policy-classifier\outputs\fting_I_om\checkpoint-107)... Done. 5.7s
wandb: Adding directory to artifact (c:\Users\allie\Documents\GitHub\policy-classifier\outputs\fting_I_om\checkpoint-214)... Done. 7.5s


In [ ]:
for r in exps:
    output_dir = cwd+"/outputs/fting_K_om"
    ds = DatasetDict.load_from_disk(f"{input_dir}/ds_{r}_om")
    hyper = {
        "epochs":10, 
        "r":r, 
        "lr":1E-5,
        "batch_size":8,
        "loss":[0.175,8.81,2.57,2.78,25.3,10.7,3.32],
        "oversampling":None
        }
    torch.cuda.empty_cache()
    gc.collect()
    wandbrun = wandb.init(config=hyper, group="K", name=f"K{r}")
    metrics = finetune_roberta(ds, int2label_dct["om"], label2int_dct["om"], "om", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper, report_to="wandb")
    wandbrun.finish()
    metriclog[f'om_{r}'] = metrics
    hyp_rpt = {"om":hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)

for r in exps:
    output_dir = cwd+"/outputs/fting_L_om"
    ds = DatasetDict.load_from_disk(f"{input_dir}/ds_{r}_om")
    hyper = {
        "epochs":10, 
        "r":r, 
        "lr":1E-5,
        "batch_size":8,
        "loss":None,
        "oversampling":"auto"
        }
    torch.cuda.empty_cache()
    gc.collect()
    wandbrun = wandb.init(config=hyper, group="L", name=f"L{r}")
    metrics = finetune_roberta(ds, int2label_dct["om"], label2int_dct["om"], "om", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper, report_to="wandb")
    wandbrun.finish()
    metriclog[f'om_{r}'] = metrics
    hyp_rpt = {"om":hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)

## Evaluation 
(deprecated, now use display_model classes to evaluate runs)

Then test the model by using it to predict labels, or by using it as an embedder with an external classifier that predicts labels

In [9]:
from classifier.ft_classification import run_experiments, modelpred_dsdct_clsf, svm_dsdct_clsf
from classifier.run_classifiers import res_dct_to_cls_rpt, cls_rpt_to_exp_rpt
from classifier.finetune import load_labelintdcts
from tqdm import tqdm
int2label_dct, label2int_dct = load_labelintdcts()
outfn = "13Apr"
#output_dir = cwd+"/outputs/fting_G_bn"
output_dir = cwd+"/outputs/fting_J_mc"

models = {}
for i in exps:#[0,3,6,9]:#range(10):
    #models[f'bert_bn_e5_r{i}'] = output_dir+f'/paraphrase-xlm-r-multilingual-v1_bn_e5_r{i}.pt'
    #models[f'bert_bn_e10_r{i}'] = output_dir+f'/paraphrase-xlm-r-multilingual-v1_bn_e10_r{i}.pt'
    models[f'bert_mc_e15_r{i}'] = output_dir+f'/paraphrase-xlm-r-multilingual-v1_mc_e15_r{i}.pt'
print(list(models))
model_res_dict = {'bn':{}, 'mc':{}}
svm_res_dict = {'bn':{}, 'mc':{}}
stw = time.time()
for model in tqdm(models):
    print(f'\nRunning model {model}')
    #print(torch.cuda.memory_allocated() / 1e9, "GB allocated before model")
    #print(torch.cuda.memory_reserved() / 1e9, "GB reserved before model")
    st = time.time()
    mode = model.split('_')[1]
    torch.cuda.empty_cache()
    if mode=='bn':
        r = model.split('_')[-1][1:]
        if r == "h":
            r = model.split('_')[-2][1:]
        try:
            dsdct = DatasetDict.load_from_disk(input_dir+f"/ds_{r}_bn")
            model_res_dict[mode][model] = modelpred_dsdct_clsf(dsdct, int2label_dct[mode], label2int_dct[mode], models[model], cuda=True)
            svm_res_dict[mode][model] = svm_dsdct_clsf(dsdct, models[model], cuda=True)
        except Exception as e:
            print(f"\nError in {model}: {e}\n")
    else:
        r = model.split('_')[-1][1:]
        try:
            dsdct = DatasetDict.load_from_disk(input_dir+f"/ds_{r}_mc")
            model_res_dict[mode][model] = modelpred_dsdct_clsf(dsdct, int2label_dct[mode], label2int_dct[mode], models[model], cuda=True)
            svm_res_dict[mode][model] = svm_dsdct_clsf(dsdct, models[model], cuda=True)
        except Exception as e:
            print(f"\nError in {model}: {e}\n")
    print(f"{model} run completed in in {round(time.time()-st,2)}s")
    torch.cuda.empty_cache()
    #gc.collect()
etw = time.time()-stw
print("Time elapsed total:", etw//60, "min and", round(etw%60), "sec")

NameError: name 'exps' is not defined

In [ ]:
with open(f"{output_dir}/randp_{outfn}_model.json", 'w', encoding="utf-8") as outfile:
    json.dump(model_res_dict, outfile, ensure_ascii=False, indent=4)
with open(f"{output_dir}/randp_{outfn}_svm.json", 'w', encoding="utf-8") as outfile:
    json.dump(svm_res_dict, outfile, ensure_ascii=False, indent=4)
mdl_cls_rpt = res_dct_to_cls_rpt(model_res_dict, int2label_dct)
mdl_exp_rpt = cls_rpt_to_exp_rpt(mdl_cls_rpt)
with open(f"{output_dir}/exprpt_{outfn}_mdl.json", 'w', encoding="utf-8") as outfile:
    json.dump(mdl_exp_rpt, outfile, ensure_ascii=False, indent=4)
svm_cls_rpt = res_dct_to_cls_rpt(svm_res_dict, int2label_dct)
svm_exp_rpt = cls_rpt_to_exp_rpt(svm_cls_rpt)
with open(f"{output_dir}/exprpt_{outfn}_svm.json", 'w', encoding="utf-8") as outfile:
    json.dump(svm_exp_rpt, outfile, ensure_ascii=False, indent=4)

print(f"finished {outfn}")

## Non fine-tuned

In [10]:
from sklearn import svm
from sentence_transformers import SentenceTransformer

def encode_all_sents(all_sents, sbert_model):
    '''
    modified from previous repository's latent_embeddings_classifier.py
    '''
    stacked = np.vstack([sbert_model.encode(sent) for sent in tqdm(all_sents)])
    return [torch.from_numpy(element).reshape((1, element.shape[0])) for element in stacked]

def nonft_roberta(dsdct, mode, cls_mode="svm", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=f"{os.getcwd()}/outputs/models", hyperparams=False):
    '''
    '''
    if not hyperparams:
        hyperparams = {
            "epochs":0, 
            "r":9,
            "oversampling":False
            }
    epochs = hyperparams["epochs"]
    rstate = hyperparams["r"]
    ovs_ratio = hyperparams["oversampling"]
    start = time.time()
    print(f'\nLoading model {model_name}\n')
    train_sents = dsdct["train"]["text"]+dsdct["test"]["text"]
    train_labels = dsdct["train"]["label"]+dsdct["test"]["label"]
    model = SentenceTransformer(model_name, device=dev)
    if ovs_ratio:
        ros = RandomOverSampler(sampling_strategy='auto', random_state=rstate)
        train_texts_resampled, train_labels_resampled = ros.fit_resample(np.array(train_sents).reshape(-1, 1), np.array(train_labels))
        train_texts_resampled, train_labels = shuffle(train_texts_resampled, train_labels_resampled, random_state=rstate)
        train_sents = list(train_texts_resampled.flatten())
    train_embs = encode_all_sents(train_sents, model)
    print("Encoding test sentences.")
    test_embs = encode_all_sents(dsdct["holdout"]["text"], model)
    if cls_mode == "svm":
        clf = svm.SVC(gamma=0.001, C=100., random_state=rstate)
        clf.fit(np.vstack(train_embs), train_labels)
        prds_lst = [int(clf.predict(sent_emb)[0]) for sent_emb in test_embs]
    accuracy = evaluate.load("accuracy")
    f1 = evaluate.load("f1")
    recall = evaluate.load("recall")
    metric_log = []
    #
    metrics = {
            "accuracy": accuracy.compute(predictions=prds_lst, references=dsdct["holdout"]["label"])["accuracy"],
            "f1": f1.compute(predictions=prds_lst, references=dsdct["holdout"]["label"], average="weighted")["f1"],
            "recall": recall.compute(predictions=prds_lst, references=dsdct["holdout"]["label"], average="weighted")["recall"]
        }
    metric_log.append(metrics)
    print("Training")
    print("Saving")
    model_fn = f"{model_name.split('/')[-1]}_{mode}_e{epochs}_r{rstate}.pt"
    model.save_pretrained(output_dir+f"/{model_fn}")
    with open(output_dir+f"/{model_fn}/metrics.json", "w", encoding="utf-8") as f:
        json.dump(metric_log, f, ensure_ascii=False, indent=4)
    end = time.time()
    print(metrics)
    print(f"\nSaved {model_name.split('/')[-1]}_{mode}_e{epochs}_r{rstate}.")
    print(f'\nDone in {round((end-start)/60,2)} min')
    return metrics

In [16]:
exps = range(10)
for r in exps:
    letter = "L"
    mode = "bn"
    output_dir = cwd+f"/outputs/fting_{letter}_{mode}"
    hyper = {
        "epochs":0, 
        "r":r,
        "oversampling":True
        }
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    ds = DatasetDict.load_from_disk(f"{input_dir}/ds_{r}_{mode}")
    torch.cuda.empty_cache()
    gc.collect()
    metrics = nonft_roberta(ds, mode, cls_mode="svm", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper)
    hyp_rpt = {mode:hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)


Loading model sentence-transformers/paraphrase-xlm-r-multilingual-v1



100%|██████████| 1850/1850 [00:24<00:00, 76.24it/s]


Encoding test sentences.


100%|██████████| 284/284 [00:03<00:00, 75.52it/s]


Training
Saving
{'accuracy': 0.8450704225352113, 'f1': 0.8450704225352113, 'recall': 0.8450704225352113}

Saved paraphrase-xlm-r-multilingual-v1_bn_e0_r0.

Done in 0.63 min

Loading model sentence-transformers/paraphrase-xlm-r-multilingual-v1



...

Training
Saving
{'accuracy': 0.8556338028169014, 'f1': 0.8551022477557931, 'recall': 0.8556338028169014}

Saved paraphrase-xlm-r-multilingual-v1_bn_e0_r8.

Done in 0.71 min

Loading model sentence-transformers/paraphrase-xlm-r-multilingual-v1



100%|██████████| 1850/1850 [00:25<00:00, 72.33it/s]


Encoding test sentences.


100%|██████████| 284/284 [00:03<00:00, 75.29it/s]


Training
Saving
{'accuracy': 0.8204225352112676, 'f1': 0.825665597385059, 'recall': 0.8204225352112676}

Saved paraphrase-xlm-r-multilingual-v1_bn_e0_r9.

Done in 0.87 min


In [17]:
for r in exps:
    letter = "M"
    mode = "mc"
    output_dir = cwd+f"/outputs/fting_{letter}_{mode}"
    hyper = {
        "epochs":0, 
        "r":r,
        "oversampling":True
        }
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    ds = DatasetDict.load_from_disk(f"{input_dir}/ds_{r}_{mode}")
    torch.cuda.empty_cache()
    gc.collect()
    metrics = nonft_roberta(ds, mode, cls_mode="svm", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper)
    hyp_rpt = {mode:hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)

for r in exps:
    letter = "N"
    mode = "om"
    output_dir = cwd+f"/outputs/fting_{letter}_{mode}"
    hyper = {
        "epochs":0, 
        "r":r,
        "oversampling":True
        }
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    ds = DatasetDict.load_from_disk(f"{input_dir}/ds_{r}_{mode}")
    torch.cuda.empty_cache()
    gc.collect()
    metrics = nonft_roberta(ds, mode, cls_mode="svm", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper)
    hyp_rpt = {mode:hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)


Loading model sentence-transformers/paraphrase-xlm-r-multilingual-v1



100%|██████████| 378/378 [00:04<00:00, 76.29it/s]


Encoding test sentences.


100%|██████████| 53/53 [00:00<00:00, 74.82it/s]


Training
Saving
{'accuracy': 0.9433962264150944, 'f1': 0.9424366963259234, 'recall': 0.9433962264150944}

Saved paraphrase-xlm-r-multilingual-v1_mc_e0_r0.

Done in 0.54 min

Loading model sentence-transformers/paraphrase-xlm-r-multilingual-v1



...

Training
Saving
{'accuracy': 0.9154929577464789, 'f1': 0.9105631255215358, 'recall': 0.9154929577464789}

Saved paraphrase-xlm-r-multilingual-v1_om_e0_r8.

Done in 1.7 min

Loading model sentence-transformers/paraphrase-xlm-r-multilingual-v1



100%|██████████| 6475/6475 [01:33<00:00, 69.08it/s]


Encoding test sentences.


100%|██████████| 284/284 [00:04<00:00, 65.47it/s]


Training
Saving
{'accuracy': 0.8556338028169014, 'f1': 0.8453005677083092, 'recall': 0.8556338028169014}

Saved paraphrase-xlm-r-multilingual-v1_om_e0_r9.

Done in 1.8 min
